In [1]:
import pymysql
import pandas as pd
import sys
import copy

In [2]:
PRICE = [[0, 100000, 700001, sys.maxsize], [100001, 300000], [300001, 500000], [500001, 700000]]

In [3]:
BODY = [['Hatchback', 'Liftback'], ['Sedan', 'Targa', 'Coupe', 'Convertible'], ['MPV', 'Wagon'], ['SUV', 'Crossover', 'Pickup']]

In [4]:
FUEL = [['Petrol'], ['Diesel']]

In [5]:
AGE = [0, 3], [4, 6], [7, 9], [10, sys.maxsize]

In [6]:
TRANSMISSION = [['AMT', 'AUTOMATIC'], ['Manual']]

In [7]:
def permutations(arrays, i=0):
    if i == len(arrays):
        return [[]]
    
    res_next = permutations(arrays, i+1)
    res = []
    for n in arrays[i]:
        for arr in res_next:
            res.append([n] + arr)
    
    return res 

In [8]:
permutations = permutations([PRICE, BODY, FUEL, AGE, TRANSMISSION])

In [9]:
permutations

[[[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [0, 3],
  ['AMT', 'AUTOMATIC']],
 [[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [0, 3],
  ['Manual']],
 [[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [4, 6],
  ['AMT', 'AUTOMATIC']],
 [[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [4, 6],
  ['Manual']],
 [[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [7, 9],
  ['AMT', 'AUTOMATIC']],
 [[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [7, 9],
  ['Manual']],
 [[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [10, 9223372036854775807],
  ['AMT', 'AUTOMATIC']],
 [[0, 100000, 700001, 9223372036854775807],
  ['Hatchback', 'Liftback'],
  ['Petrol'],
  [10, 9223372036854775807],
  ['Manual']],
 [[0, 100000, 700001, 9223

In [25]:
def sql_result(query):
    database = pymysql.connect(host='db-read.truebil.com', user='truebil-read',
                               passwd='newreadpassword', db='truebil')
    cursor = database.cursor()
    cursor.execute(query)
    database.commit()
    query_result_list = [row for row in cursor.fetchall()]
    try:
        field_names = [i[0] for i in cursor.description]
        query_result_dataframe = pd.DataFrame(data=query_result_list, columns=field_names)
    except:
        query_result_dataframe = None
    return query_result_dataframe

In [26]:
DATA_QUERY = '''SELECT sl.listing_id, (YEAR(sl.token_date) - YEAR(l.manufacturing_date)) AS age,
l.body_type_id AS body_type, l.fuel_type_id_primary AS fuel_type, l.transmission_type_id AS
transmission_type, l.price AS price FROM sold_listings sl JOIN listings l ON sl.listing_id = l.id
WHERE l.body_type_id IS NOT NULL AND l.price > 1000'''

In [27]:
data = sql_result(DATA_QUERY)

In [28]:
def get_cluster_data(c, mask_list):
    d = copy.deepcopy(c)
    for p in mask_list:
        d = d[p]
    if (len(d.index) >= 20):
        return d, len(mask_list)
    elif (len(mask_list) > 1):
        del mask_list[-1]
        k, l = get_cluster_data(c, mask_list)
        return k, l

In [29]:
for permutation in permutations:
    if len(permutation[0]) == 4:
        price_mask = (data['price'] <= permutation[0][1]) | (data['price'] >= permutation[0][2])
    else:
        price_mask = (data['price'] >= permutation[0][0]) & (data['price'] <= permutation[0][1])
    body_type_mask = (data['body_type'].isin(permutation[1]))
    fuel_type_mask = (data['fuel_type'].isin(permutation[2]))
    age_mask = (data.age >= permutation[3][0]) & (data.age <= permutation[3][1])
    transmission_type_mask = (data['transmission_type'].isin(permutation[4]))
    
    cluster_data = data[price_mask & body_type_mask & fuel_type_mask & age_mask & transmission_type_mask]
    if len(cluster_data.index) < 20:
        mask_list = [price_mask, body_type_mask, fuel_type_mask, age_mask]
        cluster_data, index = get_cluster_data(data, mask_list)
    
    #print(len(cluster_data))


/home/pavank/miniconda3/envs/dsenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [30]:
import collections
def instantiate_multi_index_dataframe():
    price_slabs = collections.OrderedDict(
        [('(0, 1)', (0, 1)), ('(1, 3)', (1, 3)), ('(3, 7)', (3, 7)), ('(7, 50)', (7, 50))])

    demand = ['High_Demand', 'Mid_Demand', 'Low_Demand']

    price_slabs_keys = list(price_slabs.keys())[1:]

    stats_metric = ['count', 'mean', 'std']
    cols = pd.MultiIndex.from_product([demand, price_slabs_keys, stats_metric],
                                      names=['model', 'price_slabs_keys', 'metric'])
    return cols, price_slabs

In [31]:
cols, price_slabs = instantiate_multi_index_dataframe()

In [32]:
SOLD_DATA_QUERY = """SELECT l.id, 
                                  l.price, 
                                  lo.city_id, 
                                  v.model_id 
                            FROM   listings l 
                                  JOIN variants v 
                                        ON l.variant_id = v.id 
                                  JOIN localities lo 
                                        ON l.locality_id = lo.id 
                            WHERE  l.is_inventory = 1 
                                  AND l.created_at >= '{start}' 
                                  AND l.status IN ( 'truebiled' ) 
                                  AND l.price > 10000"""

In [98]:
import datetime, dateutil
sold_data = sql_result(SOLD_DATA_QUERY.format(start=datetime.date.today() - dateutil.relativedelta.relativedelta(years=1)))

In [99]:
def get_active_days_demand(ids):
    list_all_aDu = activeDuration(ids)
    list_all_aDa = activeDates(list_all_aDu)

    List_Leads = lead_genarated(ids, WTS)

    cou = 0
    for i in range(len(WTS)):
        if WTS[i] != 0:
            countLeadsPerDate(List_Leads[cou], list_all_aDa, wt=WTS[i])
            cou += 1

    return ([list_all_aDu, list_all_aDa])


def get_sold_cars_active_dates(sold_cars_df, price_changes_done):
    [sold_data_aDu, sold_data_aDa] = get_active_days_demand(sold_cars_df['id'].tolist())

    sold_data_aDa['latest_dates'] = "{}"

    # get the active days of the last active price
    for row in sold_data_aDa.itertuples():
        count = 0
        sold_data_aDa.loc[row.Index]['latest_dates'] = dict()
        try:
            for date in row.dates.keys():
                if date > max(sold_data_aDa.loc[row.Index]['start_date'],
                              price_changes_done.loc[row.Index]['start_date']):
                    count += 1
                    sold_data_aDa.loc[row.Index]['latest_dates'][date] = row.dates[date]
        except Exception as e:
            sold_data_aDa.loc[row.Index]['latest_dates'] = row.dates
    return sold_data_aDa


In [ ]:
price_changes_done = get_price_changes(PRICE_CHANGES_QUERY.format(condition=" DATE(l.created_at) NOT IN ('2019-11-18', '2019-10-11', '2019-06-19') "))
sold_data_aDa = get_sold_cars_active_dates(sold_cars_df=sold_data, price_changes_done=price_changes_done)

In [1]:
import itertools

In [2]:
l = [[300001, 500000], ['Sedan', 'Targa', 'Coupe', 'Convertible'], ['Petrol'], [7, 9], ['Manual']]

In [7]:
l = [[[1, 2, 3], [2, 4]]]

In [8]:
li = list(itertools.chain.from_iterable(l))

In [9]:
li

[[1, 2, 3], [2, 4]]

In [5]:
cluster_string = str(l[0][0]) + ' < Price < '+ str(li[0][1]) + ', Body: ' + li[2] + ', Age: ' + str(li[3])

TypeError: 'int' object is not subscriptable

In [6]:
', '.join(map(str, li))

'300001, 500000, Sedan, Targa, Coupe, Convertible, Petrol, 7, 9, Manual'